In [ ]:
import pandas as pd
import numpy as np
import matplotlib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from spellchecker import SpellChecker
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from collections import defaultdict
import string
import re

STOPWORDS = set(stopwords.words('english'))
pd.set_option('display.max_colwidth', -1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [ ]:
train = pd.read_csv('data/train_pre_processing.csv')
test = pd.read_csv('data/test_pre_processing.csv')
train_ID = pd.read_csv('data/train.csv') #para agregar el id a train que no posee
train.info()

## Text Cleaning

In [ ]:
def remove_url(text):
    text = re.compile(r'https?://\S+|www\.\S+').sub(r' ', text)
    text = re.compile(r'http\S+').sub(r'', text)
    text = re.compile(r'www\S+').sub(r'', text)
    text = re.compile(r'pic.twitter.com\S+').sub(r' ', text)
    return text

In [ ]:
def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r' ',text)

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
def remove_punctuation(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

In [ ]:
def remove_stopwords(text):
    words = word_tokenize(text)
    words = [w for w in words if not w in STOPWORDS]
    return ' '.join(words)

In [ ]:
spell = SpellChecker()
def spellcheck(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

In [ ]:
def clean_non_ascii(text):
    return re.compile(r'[^A-Za-z0-9\.\'!\?,\$]').sub(r' ', text)

In [ ]:
def clean_spaces(text):
    # Reemplazar multiples espacios en uno
    return re.sub('\s{2,}', ' ', text)

In [ ]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}
def convert_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word

def convert_abbrev_in_text(text):
    tokens = word_tokenize(text)
    tokens = [convert_abbrev(word) for word in tokens]
    text = ' '.join(tokens)
    return text

In [ ]:
def clean_text(text):
    text = text.lower()
    text = remove_url(text)
    text = remove_html(text)
    text = remove_emoji(text)
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    #text = clean_non_ascii(text)
    #text = clean_spaces(text)
    #text = convert_abbrev_in_text(text)
    #text = spellcheck(text)
    return text

In [ ]:
train['text_clean'] = train.text.transform(lambda x: clean_text(str(x)))
test['text_clean'] = test.text.transform(lambda x: clean_text(str(x)))

In [ ]:
train[['text','text_clean']].sample(2)

,text,text_clean
2591,Black Eye 9: A space battle occurred at Star O784 involving 3 fleets totaling 3942 ships with 14 destroyed,black eye 9 space battle occurred star o784 involving 3 fleets totaling 3942 ships 14 destroyed
354,Salvation Army hosts rally to reconnect fathers with children: The Salvation Army is hosting a back to school rallyÛ_ http://t.co/rDjpor3AZg,salvation army hosts rally reconnect fathers children salvation army hosting back school rallyû


## Mean Word Length

In [ ]:
train['mean_word_len'] = train.text.transform(lambda x: np.mean([len(word) for word in str(x).split()]))
test['mean_word_len'] = test.text.transform(lambda x: np.mean([len(word) for word in str(x).split()]))

## N-gram

In [ ]:
def get_ngrams(text, n_gram=1):
    token = []
    words = word_tokenize(text)
    for w in words:
        filter_words = STOPWORDS.union(set(string.punctuation))
        if w not in filter_words:
            token.append(w)
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [' '.join(ngram) for ngram in ngrams]

## Bigrams

In [ ]:
disaster_bigrams = defaultdict(int)
nondisaster_bigrams = defaultdict(int)

for tweet in train.loc[train['target'] == 1]['text_clean']:
    for word in get_ngrams(tweet, 2):
        disaster_bigrams[word] += 1
        
for tweet in train.loc[train['target'] == 0]['text_clean']:
    for word in get_ngrams(tweet, 2):
        nondisaster_bigrams[word] += 1

In [ ]:
# Nos mantenemos con los que tengan una frecuencia mayor o igual a 10 en desastre
delete_keys = [k for k, v in disaster_bigrams.items() if v < 10]
for key in delete_keys:
    del disaster_bigrams[key]

delete_keys = [k for k, v in nondisaster_bigrams.items() if v < 10]
for key in delete_keys:
    del nondisaster_bigrams[key]

In [ ]:
# Feature booleano para saber si el texto tiene un bigrama de desastre o no

train['text_disaster_bigram'] = train.text_clean.transform(lambda x: any(disngram in disaster_bigrams.keys() for disngram in get_ngrams(x, 2)))
train['text_nondisaster_bigram'] = train.text_clean.transform(lambda x: any(disngram in nondisaster_bigrams.keys() for disngram in get_ngrams(x, 2)))
test['text_disaster_bigram'] = test.text_clean.transform(lambda x: any(disngram in disaster_bigrams.keys() for disngram in get_ngrams(x, 2)))
test['text_nondisaster_bigram'] = test.text_clean.transform(lambda x: any(disngram in nondisaster_bigrams.keys() for disngram in get_ngrams(x, 2)))

In [ ]:
train['text_disaster_bigram'].value_counts()

False    6346
True     1267
Name: text_disaster_bigram, dtype: int64

In [ ]:
train['text_nondisaster_bigram'].value_counts()

False    6749
True     864 
Name: text_nondisaster_bigram, dtype: int64

## Trigrams

In [ ]:
disaster_trigrams = defaultdict(int)
nondisaster_trigrams = defaultdict(int)

for tweet in train.loc[train['target'] == 1]['text_clean']:
    for word in get_ngrams(tweet, 3):
        disaster_trigrams[word] += 1
        
for tweet in train.loc[train['target'] == 0]['text_clean']:
    for word in get_ngrams(tweet, 2):
        nondisaster_trigrams[word] += 1

In [ ]:
# Nos mantenemos con los que tengan una frecuencia mayor o igual a 3
delete_keys = [k for k, v in disaster_trigrams.items() if v < 3]
for key in delete_keys:
    del disaster_trigrams[key]

delete_keys = [k for k, v in nondisaster_trigrams.items() if v < 3]
for key in delete_keys:
    del nondisaster_trigrams[key]

In [ ]:
# Feature booleano para saber si el texto tiene un bigrama de desastre o no

train['text_disaster_trigram'] = train.text_clean.transform(lambda x: any(disngram in disaster_trigrams.keys() for disngram in get_ngrams(x, 3)))
train['text_nondisaster_trigram'] = train.text_clean.transform(lambda x: any(disngram in nondisaster_trigrams.keys() for disngram in get_ngrams(x, 2)))
test['text_disaster_trigram'] = test.text_clean.transform(lambda x: any(disngram in disaster_trigrams.keys() for disngram in get_ngrams(x, 3)))
test['text_nondisaster_trigram'] = test.text_clean.transform(lambda x: any(disngram in nondisaster_trigrams.keys() for disngram in get_ngrams(x, 2)))

In [ ]:
train['text_disaster_trigram'].value_counts()

False    6384
True     1229
Name: text_disaster_trigram, dtype: int64

In [ ]:
train['text_nondisaster_trigram'].value_counts()

False    5462
True     2151
Name: text_nondisaster_trigram, dtype: int64

## TF IDF

In [ ]:
vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    norm='l2',
    min_df=0,
    smooth_idf=False,
    max_features=5000)
X = vectorizer.fit_transform(train['text_clean'])

In [ ]:
df_tfidf = pd.DataFrame(X.todense(), columns = vectorizer.get_feature_names())
df_tfidf.drop(labels = ['location','text', 'target'], axis=1, inplace=True)
train = train.join(df_tfidf)
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Columns: 5041 entries, keyword to zouma
dtypes: bool(8), float64(5000), int64(27), object(6)
memory usage: 292.4+ MB
None


In [ ]:
X = vectorizer.transform(test['text_clean'])
df_tfidf = pd.DataFrame(X.todense(), columns = vectorizer.get_feature_names())
df_tfidf.drop(labels = ['id','location','text', 'target'], axis=1, inplace=True)
test = test.join(df_tfidf)
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Columns: 5056 entries, id to zouma
dtypes: bool(21), float64(4999), int64(31), object(5)
memory usage: 125.4+ MB
None


In [ ]:
target = train.target
train.drop(columns=['target'], inplace=True)
train['target'] = target
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Columns: 5041 entries, keyword to target
dtypes: bool(8), float64(5000), int64(27), object(6)
memory usage: 292.4+ MB


In [ ]:
train.head(2)

,keyword,location,text,keyword_grouped,text_contain_keyword,total_words,len_text,total_upper_chars,total_numbers_chars,total_special_chars,total_common_chars,contain_question,contain_link,contain_hashtag,contain_upper_words,total_3_words,total_4_words,total_5_words,total_6_words,total_7_words,total_8_words,total_3_ormore_words,total_4_ormore_words,total_5_ormore_words,total_6_ormore_words,total_7_ormore_words,total_8_ormore_words,total_3_orless_words,total_4_orless_words,total_5_orless_words,total_6_orless_words,total_7_orless_words,total_8_orless_words,subjectivity_text,polarity_text,stopword_count,unique_word_count,text_clean,mean_word_len,text_disaster_bigram,...,ya,yahoo,yahoonews,yall,yay,yazidis,yea,yeah,year,years,yellow,yemen,yep,yes,yesterday,yo,yobe,york,yorker,youd,youll,young,youngheroesid,youre,youth,youtube,youve,yr,yrs,yugvani,yyc,z,z10,zakbagans,zayn,zionist,zombie,zone,zouma,target
0,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,NaN,NaN,13,69,10,0,1,12,False,False,True,True,5,1,2,1,1,0,11,6,5,3,2,1,7,8,10,11,12,12,0.0,0.0,5,13,deeds reason earthquake may allah forgive us,4.384615,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,NaN,NaN,Forest fire near La Ronge Sask. Canada,NaN,NaN,7,38,5,0,0,7,False,False,False,False,0,2,2,2,0,0,6,6,4,2,0,0,1,3,5,7,7,7,0.4,0.1,0,7,forest fire near la ronge sask canada,4.571429,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


## CREANDO EL CSV

In [ ]:
def norm_true_and_false(x):
  if x is False:
    return 0
  return 1

In [ ]:
test_cnnd=pd.DataFrame()
train_cnnd=pd.DataFrame()

In [ ]:
train["id"]=train_ID.id
test["len_text"]=test.text.apply(lambda x: len(x))

In [ ]:
train_cnn["id"]=train.id
train_cnn["text_clean"]=train.text_clean
train_cnn["target"]=train.target
train_cnn["len_text"]=train.len_text
train_cnn["count_word_uniques"]=train.total_words
train_cnn["text_contain_keyword_norm"]=train.text_contain_keyword.apply(lambda x:norm_true_and_false(x))
train_cnn["contain_link_norm"]=train.contain_link.apply(lambda x:norm_true_and_false(x))
train_cnn["contain_question_norm"]=train.contain_question.apply(lambda x:norm_true_and_false(x))
train_cnn["contain_hashtag_norm"]=train.contain_hashtag.apply(lambda x:norm_true_and_false(x))

In [ ]:
test_cnn["id"]=test.id
test_cnn["text_clean"]=test.text_clean
test_cnn["len_text"]=test.len_text
test_cnn["count_word_uniques"]=test.total_words
test_cnn["text_contain_keyword_norm"]=test.text_contain_keyword.apply(lambda x:norm_true_and_false(x))
test_cnn["contain_link_norm"]=test.contain_link.apply(lambda x:norm_true_and_false(x))
test_cnn["contain_question_norm"]=test.contain_question.apply(lambda x:norm_true_and_false(x))
test_cnn["contain_hashtag_norm"]=test.contain_hashtag.apply(lambda x:norm_true_and_false(x))

In [ ]:
train.to_csv('data/train_pre_processing_nlp_5000.csv', index=False)
test.to_csv('data/test_pre_processing_nlp_5000.csv', index=False)
train_cnn.to_csv('data/train_cnn.csv', index=False)
test_cnn.to_csv('data/train_cnn.csv', index=False)